In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    'post_padding', 'pre_padding', 'mid_padding', 
                 'strf_padding', 'ext_padding', 
                 'rnd_padding', 'aug_padding', "zoom_padding"]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["zoom_padding"]
#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/1conv/64filts_size5/'#'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 2 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [314,77, 8]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 64 #None
kernel_size = [5] #None
pool_size = None
n_hid = None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task1/"
#architecture = "only_denses"
architecture = "conv_dense"

In [3]:
lr=0.0001
optimizer = Adam(lr=lr)

### Loading stuff

In [4]:
file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_split.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_split.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model  - Task 1

In [5]:
#model = building_3dense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
model = building_1convdense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task1', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 314)         20410     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 314)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 77)          24255     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 77)          0         
__________

### Training model

In [6]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, val_generator,
                        architecture, max_lenn, dict_size, batch_size,
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val, 
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
dense_5 (Dense)              (None, 1000, 314)         20410     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 314)         0         
_________________________________________________________________
dense_6 (Dense)              (None, 1000, 77)          24255     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1000, 77)          0         
__________

KeyboardInterrupt: 